In [1]:
import cv2 as cv
import time

In [2]:
# Leemos los clasificadores pre-entrenados (instalados en cv2/data/haarcascades/)
#--------------------------------------------------------------------------------
face_cascade = cv.CascadeClassifier('C:\haarcascades\haarcascade_frontalface_default.xml')
eye_cascade = cv.CascadeClassifier('C:\haarcascades\haarcascade_eye.xml')
smile_cascade = cv.CascadeClassifier('C:\haarcascades\haarcascade_smile.xml')

In [ ]:
# Cargamos la imagen a analizar
#------------------------------
originalImage = cv.imread('Rollings.jpg')
cap = cv.VideoCapture(0)
fourcc = cv.VideoWriter_fourcc(*'MP4V')
out = cv.VideoWriter('output.mp4', fourcc, 20.0, (640,480))
while(cap.isOpened()):
  # Tomamos un nuevo frame
  #-----------------------
    ret ,originalImage = cap.read()
    if ret == True:
      # Pasamos la imagen a escala de grises
      #-------------------------------------
      grayImage = cv.cvtColor(originalImage, cv.COLOR_BGR2GRAY)
      grayImage = cv.equalizeHist(grayImage)

      # Llamamos al clasificador de Haar (AdaBoost)
      #--------------------------------------------
      #faces = face_cascade.detectMultiScale(grayImage, 1.012, 5)
      #faces = face_cascade.detectMultiScale(grayImage, 1.1, 1)#, minSize= (100,130),maxSize=(200,400))
      faces = face_cascade.detectMultiScale(grayImage,scaleFactor= 1.1, minNeighbors=5,minSize= (100,130), maxSize=(400,400))
      # Recorro las caras encontradas
      #------------------------------
      for (x,y,w,h) in faces:
          # Le dibujamos un rectángulo amarillo
          cv.rectangle(originalImage,(x,y),(x+w,y+h),(255,255,0),2)
          # Definimos las ROIs en la imagen gris y color
          roi_gray = grayImage[y:y+h, x:x+w] 
          roi_color = originalImage[y:y+h, x:x+w] 
          # Para cada rostro hallado le buscamos los ojos
          eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor = 1.1, minNeighbors =3,minSize = (30,30),maxSize = (60,60))
          #eyes = eye_cascade.detectMultiScale(roi_gray, 
           #                     scaleFactor= 1.1,
           #                     minNeighbors=3,
           #                     minSize = (32,32),
           #                     maxSize = (64,64))  
          # En los ojos hallados les dibujamos rectángulos
          for (ex,ey,ew,eh) in eyes:
              cv.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
          smiles = smile_cascade.detectMultiScale(roi_gray, 1.8, 23)
               
          for (sx, sy, sw, sh) in smiles:
             cv.rectangle(roi_color,(sx, sy),((sx + sw),(sy + sh)),(0, 0, 255), 2)
      out.write(originalImage)          
      cv.imshow('Detección de caras con filtros de Haar en cascada',originalImage) 
      #cv.waitKey(0)
      #cv.destroyAllWindows()

      #time.sleep(10) 
      #cv.destroyAllWindows()

      k = cv.waitKey(30) & 0xff
      if k == 27:
         break
    else:
      break
out.release()
cv.destroyAllWindows()
cap.release()